In [ ]:
import networkx as nx 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sys
import seaborn as sns
import json
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.tokenize import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS

#This section imports all libraries to be used

data1 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=1&api_key=RGAPI-a6bbc195-66e2-4f9c-b136-938d292db3e6"
                  )
data2 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=2&api_key=RGAPI-a6bbc195-66e2-4f9c-b136-938d292db3e6"
                  )
data3 = pd.read_json(
    'https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=3&api_key=RGAPI-a6bbc195-66e2-4f9c-b136-938d292db3e6'
)

data4 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=1&api_key=RGAPI-a6bbc195-66e2-4f9c-b136-938d292db3e6"
                  )
data5 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=2&api_key=RGAPI-a6bbc195-66e2-4f9c-b136-938d292db3e6"
                  )
data6 = pd.read_json(
    'https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=3&api_key=RGAPI-a6bbc195-66e2-4f9c-b136-938d292db3e6'
)

#This accesses the API for all data

d1 = pd.DataFrame(data1)
d2 = pd.DataFrame(data2)
d3 = pd.DataFrame(data3)

d4 = pd.DataFrame(data4)
d5 = pd.DataFrame(data5)
d6 = pd.DataFrame(data6)

silver_frames = [d1, d2, d3]
gold_frames = [d4, d5, d6]

silver = pd.concat(silver_frames, ignore_index=True)
gold = pd.concat(gold_frames, ignore_index=True)

#this puts the data into the two dataframes, silver and gold, based on the different ranks, and then combines them into a total dataset in the next lines of code

frames = [silver, gold]
total = pd.concat(frames, ignore_index=True)
total

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,miniSeries
0,4ee6e716-b899-42e1-a08a-b719ccf5c2ac,RANKED_SOLO_5x5,SILVER,I,RhsdWYa0WTL21Q5dxZ6e1yLIFdLke1qfX-kZCRvKZ1-MhpF5,KMSTirpitz,45,86,86,False,False,False,True,NaN
1,add324d7-b028-4af7-b156-dc1b48489677,RANKED_SOLO_5x5,SILVER,I,L-LuEfE0GlILDM68KVvXXZScsI-ON67byoKOAIdGNB5MDe27,Lowfyz,44,36,35,False,False,False,False,NaN
2,df331365-7ffa-4fcb-81d5-b1959a848f66,RANKED_SOLO_5x5,SILVER,I,-Ga4FnlH2r5ec0JmdxnC_EBY_M1mtuhvZDv8QPFxbomChOYp,Subt3rfug3,23,38,35,False,False,False,False,NaN
3,35192393-9d6e-4b88-a3c2-6fd945899f11,RANKED_SOLO_5x5,SILVER,I,u-pTStsK71w26dj9KuUFbaTyTM2XDKyvvNw75sjxnILzu-t1,PAPA Mo PANOT,22,4,9,False,False,False,False,NaN
4,c3e3c5aa-1277-4b99-97ce-c0365af4962b,RANKED_SOLO_5x5,SILVER,I,cuoCLtlZeX3xx_iMLM0eEVwHLenm6DPAmi0RvIXkpvftm8PL,BeLikeWater04,15,68,50,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,75aa7cac-2005-425a-903e-01577b7c8f48,RANKED_SOLO_5x5,GOLD,IV,HAn-NipnYQ90jujAACuenDErbRDjllWgmrDoSfsEnonYY_Ne,General Noxian,45,31,44,False,False,False,False,NaN
1226,0523311a-e861-4f80-b96f-794d2d686d2d,RANKED_SOLO_5x5,GOLD,IV,0BjMnTe59UZlBfQcsB1qYOjtVyEZ3ImOcewNL818SO_-erOQ,LittleStoyan,1,89,81,False,False,False,True,NaN
1227,09d12105-8b6c-4d7a-a331-4d85de007220,RANKED_SOLO_5x5,GOLD,IV,vIMZnpdEFPSnvvL7P7weeyi-ZDLFVFPPkzxbgZqEvRL8QTLm,mınami,43,36,35,False,False,False,False,NaN
1228,dbda589e-4d9c-40df-870c-686244d7ae73,RANKED_SOLO_5x5,GOLD,IV,pJbhyBf7btzrlWz42ElIYcy56-is8rbQLNAEKEqhq2MMHV0E,Satellark,17,55,51,False,False,True,True,NaN


In [ ]:
#This section, I created categories relevant to analysis. For my analysis purposes, I chose to use already created columns, 'veteran' and 'freshBlood'. I also created a category of players that were at the top of the rank, Silver 1, 
# and furthur filtered it down using the numeric column, 'wins'.
new_points = total[total['leaguePoints']==100]
new_points1 = new_points[new_points['tier']=='SILVER']
new = new_points1[new_points1['wins']>40]
df1 = new.drop(['leagueId',	'queueType',	'tier', 	'rank',	'summonerId',	'wins',	'losses',	'veteran',	'inactive',	'freshBlood',	'hotStreak',	'miniSeries'], axis=1)
df1
new.count()
total_new = pd.merge(total, df1, how='outer', on='summonerName', suffixes=(' original', ' for Silver players at 100 LP with over 40 wins'))
total_new

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints original,wins,losses,veteran,inactive,freshBlood,hotStreak,miniSeries,leaguePoints for Silver players at 100 LP with over 40 wins
0,4ee6e716-b899-42e1-a08a-b719ccf5c2ac,RANKED_SOLO_5x5,SILVER,I,RhsdWYa0WTL21Q5dxZ6e1yLIFdLke1qfX-kZCRvKZ1-MhpF5,KMSTirpitz,45,86,86,False,False,False,True,NaN,NaN
1,add324d7-b028-4af7-b156-dc1b48489677,RANKED_SOLO_5x5,SILVER,I,L-LuEfE0GlILDM68KVvXXZScsI-ON67byoKOAIdGNB5MDe27,Lowfyz,44,36,35,False,False,False,False,NaN,NaN
2,df331365-7ffa-4fcb-81d5-b1959a848f66,RANKED_SOLO_5x5,SILVER,I,-Ga4FnlH2r5ec0JmdxnC_EBY_M1mtuhvZDv8QPFxbomChOYp,Subt3rfug3,23,38,35,False,False,False,False,NaN,NaN
3,35192393-9d6e-4b88-a3c2-6fd945899f11,RANKED_SOLO_5x5,SILVER,I,u-pTStsK71w26dj9KuUFbaTyTM2XDKyvvNw75sjxnILzu-t1,PAPA Mo PANOT,22,4,9,False,False,False,False,NaN,NaN
4,c3e3c5aa-1277-4b99-97ce-c0365af4962b,RANKED_SOLO_5x5,SILVER,I,cuoCLtlZeX3xx_iMLM0eEVwHLenm6DPAmi0RvIXkpvftm8PL,BeLikeWater04,15,68,50,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,75aa7cac-2005-425a-903e-01577b7c8f48,RANKED_SOLO_5x5,GOLD,IV,HAn-NipnYQ90jujAACuenDErbRDjllWgmrDoSfsEnonYY_Ne,General Noxian,45,31,44,False,False,False,False,NaN,NaN
1226,0523311a-e861-4f80-b96f-794d2d686d2d,RANKED_SOLO_5x5,GOLD,IV,0BjMnTe59UZlBfQcsB1qYOjtVyEZ3ImOcewNL818SO_-erOQ,LittleStoyan,1,89,81,False,False,False,True,NaN,NaN
1227,09d12105-8b6c-4d7a-a331-4d85de007220,RANKED_SOLO_5x5,GOLD,IV,vIMZnpdEFPSnvvL7P7weeyi-ZDLFVFPPkzxbgZqEvRL8QTLm,mınami,43,36,35,False,False,False,False,NaN,NaN
1228,dbda589e-4d9c-40df-870c-686244d7ae73,RANKED_SOLO_5x5,GOLD,IV,pJbhyBf7btzrlWz42ElIYcy56-is8rbQLNAEKEqhq2MMHV0E,Satellark,17,55,51,False,False,True,True,NaN,NaN


In [ ]:
#Here, I did a count aggregration to see if any of the silver players at the top were veterans. I found that none of them were.
total_new.groupby('veteran')[['leaguePoints for Silver players at 100 LP with over 40 wins']].count()

,leaguePoints for Silver players at 100 LP with over 40 wins
veteran,
False,7
True,0


In [ ]:
#Here, I did a sum aggregation to see the amount of wins that new players, or freshBloods, were accumulating. 
total_new.groupby('freshBlood')[['wins']].sum()

,wins
freshBlood,
False,109220
True,4584


,wins
freshBlood,
False,109220
True,4584


In [ ]:
#Here I did a mean aggregation of all the silver players at 100 lp to find how many wins they had.
total_new.groupby('leaguePoints for Silver players at 100 LP with over 40 wins')[['wins']].mean()

,wins
leaguePoints for Silver players at 100 LP with over 40 wins,
100.0,120.857143


In [ ]:
#This was a max aggregation to see the highest amount of wins held be those of veteran statuses.
total_new.groupby('veteran')[['wins']].max()

,wins
veteran,
False,983
True,1253


In [ ]:
#I created a function to take the average of the median and mean of an array. I then did an aggregation using the funciton to see the original league points of those of veteran status.

def ave_of_mid_and_mean(arr):
  return (arr.mean() + arr.median())/2 

total_new.groupby('veteran')[['leaguePoints original']].agg(ave_of_mid_and_mean)

,leaguePoints original
veteran,
False,32.127402
True,13.409091


Through this assignment, I was able to answer several of my original questions from assignment 1. Firstly, I wanted to know how many people stood in my way from ascending from Silver 1 to Gold 4. I now know that there are not many players with many wins at the top of Silver 1. From previous assignments, I know that more wins typically means more games, so the players above me tend to be newer to the tier. This was also confirmed using the freshBlood column. Another question I had was whether higher ranked players were of veteran status. This was disproven in earlier assignments, and in this one where I found that, even the top of Silver 1 was filled with newer players to the rank. These were the questions answered by this assignment, while other questions were answered by previous assignments.